In [40]:
import open3d as o3d
import numpy as np
import pandas as pd
import os
from tqdm import tqdm_notebook
import gc

Convert generated range images to ply

In [38]:
range_arr = np.load("/home/sabyasachi/Projects/ati/data/data/datasets/Carla/16beam-Data/small_map/dynamic_training_data/npy_data/8.npy")
OUT_PATH = "/home/sabyasachi/Projects/ati/data/data/datasets/Carla/16beam-Data/small_map/dynamic/8/_range_out"

In [3]:
range_arr.shape

(2827, 16, 1024, 4)

In [41]:
for pcd_idx in tqdm_notebook(range(range_arr.shape[0])):
    this_arr = np.concatenate([[range_arr[pcd_idx][:,:,0].flatten()],
                               [range_arr[pcd_idx][:,:,1].flatten()],
                               [range_arr[pcd_idx][:,:,2].flatten()]], axis=0) . T
    
    this_pcd = o3d.PointCloud()
    this_pcd.points = o3d.utility.Vector3dVector(this_arr)
    
    this_fname = str(pcd_idx + 1) + ".ply"
    this_out_path = os.path.join(OUT_PATH, this_fname)
    o3d.io.write_point_cloud(this_out_path, this_pcd, write_ascii=True)

Convert generated preprocessed range images to pcd

In [77]:
range_arr = np.load("/home/sabyasachi/Projects/ati/ati_motors/adversarial_based/training_data/small_map_dynamic_high_only/data/8.npy", allow_pickle=True)
OUT_PATH = "/home/sabyasachi/Projects/ati/data/data/datasets/Carla/16beam-Data/small_map/dynamic/8/_preprocessed_out/"

In [78]:
range_arr.shape

(2827, 2, 16, 1024)

In [79]:
def to_3d(velo):
    angles = np.linspace(0, np.pi * 2, velo.shape[-1])
    dist, z = velo[0], velo[1]
    x = np.cos(angles) * dist
    y = np.sin(angles) * dist
    out = np.stack([x,y,z], axis=0)
    return out.astype('float32')

In [93]:
for pcd_idx in tqdm_notebook(range(range_arr.shape[0])):
    img = range_arr[pcd_idx]
    img = img * 100
    frame = to_3d(img)
    frame_flat = frame.reshape((3,-1))
    this_pcd = o3d.PointCloud()
    this_arr = frame_flat.T
    this_pcd.points = o3d.utility.Vector3dVector(this_arr)
    
    this_fname = str(pcd_idx + 1) + ".ply"
    this_out_path = os.path.join(OUT_PATH, this_fname)
    o3d.io.write_point_cloud(this_out_path, this_pcd, write_ascii=True)
#     break

Visually checking 2d plots

In [115]:
import shutil
import matplotlib.pyplot as plt

In [97]:
OUT_PATH = "/home/sabyasachi/Projects/ati/data/data/datasets/Carla/16beam-Data/small_map/dynamic/8"
SAMPLES_FOLDER = "samples"
OUT_FOLDER = "_out"
RANGE_OUT_FOLDER = "_range_out"
PREPROCESSED_OUT_FOLDER = "_preprocessed_out"

PLY_OUT_PATH              = os.path.join(OUT_PATH, OUT_FOLDER)
PLY_RANGE_OUT_PATH        = os.path.join(OUT_PATH, RANGE_OUT_FOLDER)
PLY_PREPROCESSED_OUT_PATH = os.path.join(OUT_PATH, PREPROCESSED_OUT_FOLDER)

SAMPLES_PATH                  = os.path.join(OUT_PATH,     SAMPLES_FOLDER)
SAMPLES_OUT_PATH              = os.path.join(SAMPLES_PATH, OUT_FOLDER)
SAMPLES_RANGE_OUT_PATH        = os.path.join(SAMPLES_PATH, RANGE_OUT_FOLDER)
SAMPLES_PREPROCESSED_OUT_PATH = os.path.join(SAMPLES_PATH, PREPROCESSED_OUT_FOLDER)

if not os.path.exists(SAMPLES_PATH):
    os.makedirs(SAMPLES_OUT_PATH)
    os.makedirs(SAMPLES_RANGE_OUT_PATH)
    os.makedirs(SAMPLES_PREPROCESSED_OUT_PATH)
else:
    shutil.rmtree(SAMPLES_PATH)
    os.makedirs(SAMPLES_OUT_PATH)
    os.makedirs(SAMPLES_RANGE_OUT_PATH)
    os.makedirs(SAMPLES_PREPROCESSED_OUT_PATH)

In [98]:
def getint(name):
    return int(name.split('.')[0])

In [99]:
out_plys              = sorted(os.listdir(PLY_OUT_PATH), key=getint)
range_out_plys        = sorted(os.listdir(PLY_RANGE_OUT_PATH), key=getint)
preprocessed_out_plys = sorted(os.listdir(PLY_PREPROCESSED_OUT_PATH), key=getint)

In [102]:
assert len(out_plys) == len(range_out_plys) == len(preprocessed_out_plys)

In [131]:
for pcd_idx in tqdm_notebook(range(len(out_plys))):
    ply = out_plys[pcd_idx]
    
    out_ply_path = os.path.join(PLY_OUT_PATH, ply)
    out_pcd = o3d.read_point_cloud(out_ply_path)
    out_arr = np.asarray(out_pcd.points)
    plt.figure()
    plt.title("Raw ply")
    plt.scatter(out_arr[:,0], out_arr[:,1], s=0.5, color='r')
    plt.savefig(os.path.join(SAMPLES_OUT_PATH, str(pcd_idx)+'.jpg')) 
    plt.close()
    
    out_ply_path = os.path.join(PLY_RANGE_OUT_PATH, ply)
    out_pcd = o3d.read_point_cloud(out_ply_path)
    out_arr = np.asarray(out_pcd.points)
    plt.figure()
    plt.title("Range ply")
    plt.scatter(out_arr[:,0], out_arr[:,1], s=0.5, color='g')
    plt.savefig(os.path.join(SAMPLES_RANGE_OUT_PATH, str(pcd_idx)+'.jpg')) 
    plt.close()
    
    out_ply_path = os.path.join(PLY_PREPROCESSED_OUT_PATH, ply)
    out_pcd = o3d.read_point_cloud(out_ply_path)
    out_arr = np.asarray(out_pcd.points)
    plt.figure()
    plt.title("Preprocessed ply")
    plt.scatter(out_arr[:,0], out_arr[:,1], s=0.5, color='b')
    plt.savefig(os.path.join(SAMPLES_PREPROCESSED_OUT_PATH, str(pcd_idx)+'.jpg')) 
    plt.close()
#     break